<a href="https://colab.research.google.com/github/ohmkas/Carteira-Quant-FI/blob/main/Gera_Carteiras_do_Dia_dos_solicitantes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta

In [ ]:
def filtrar_if_ofertante():
  execfile("/content/drive/MyDrive/Colab Notebooks/Carteira Quant/filtrar_instituição_financeira.py")
  return (None)

filtrar_if_ofertante()
try:
  with open('/content/sample_data/Clientes do Dia/controle.json', 'rb') as fp:
    cnpjs_clientes = pickle.load(fp)
except:
  pass

def consulta_solicitantes():
  execfile("/content/drive/MyDrive/Colab Notebooks/Carteira Quant/consulta_aos_solicitantes.py")
  return (None)

consulta_solicitantes()

clientes_dia = pd.read_csv('/content/sample_data/Clientes do Dia/Clientes_do_Dia.csv')

if clientes_dia.columns[0] == 'Unnamed: 0':
    clientes_dia = clientes_dia.T
    clientes_dia.columns = clientes_dia.iloc[0]
    clientes_dia = clientes_dia.iloc[1:]
    clientes_dia.index.name = 'Carimbo de data/hora'
else:
  pass  

clientes_dia = clientes_dia.set_index('Endereço de e-mail')

clientes_dia['Em quanto o tempo você espera alcançar esse objetivo?'] = clientes_dia['Em quanto o tempo você espera alcançar esse objetivo?'].str.split(' ',expand=True)[0]

clientes_dia['Qual o valor máximo de perdas que toleraria num curto prazo?'] = clientes_dia['Qual o valor máximo de perdas que toleraria num curto prazo?'].str.split('%',expand=True)[0].replace(['Nada, não quero nenhum risco nos meus investimentos'],0)

def coleta_dados_bc(codigo_bcb): #função que coleta dados do Banco Central
    url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
    df = pd.read_json(url)
    df['data'] = pd.to_datetime(df['data'], dayfirst=True)
    df.set_index('data', inplace=True)
    return (df)
 
#https://www3.bcb.gov.br/sgspub/consultarvalores/telaCvsSelecionarSeries.paint

cdi_graf = coleta_dados_bc(12)
cdi = cdi_graf.loc[(date.today()-timedelta(252)).strftime("%Y-%m"):]


def salvar(nome_pasta=str): #Cria ess pasta caso ela não exista ou simplemente já deixa o caminho da pasta pronto
  if not os.path.exists("/content/sample_data/"+nome_pasta): 
    os.mkdir("/content/sample_data/"+nome_pasta) 
  caminho = str("/content/sample_data/"+nome_pasta) 
  print('os arquivos serão salvos em:',caminho)
  return(caminho)

salvar_em = salvar(nome_pasta='Etapa Final')


controle_parcela = {}
i = float(((1+(cdi/100)).rolling(22).apply(np.prod)-1).iloc[-1].values) #juros mensal
for cliente in clientes_dia.index:
  
  n = float(clientes_dia.loc[cliente]['Em quanto o tempo você espera alcançar esse objetivo?'])
  FFV = (((1+i)**n)-1)/i
  PMTt = float(clientes_dia.loc[cliente]['Qual o valor do seu objetivo?'])/FFV
  
  PMTr = float(clientes_dia.loc[cliente]['Quanto você investirá por mês?'])

  dif = PMTr-PMTt

  if PMTt >  PMTr:
    aux = {cliente:'objetivo não será alcançado. É preciso aumentar a parcela em '+str(round(dif,2)*-1)+
                             ' reais, ou seja, você precisa contribuir '+str(round(PMTt,2))+' reais.'}
    
  else:
    aux = {cliente:'objetivo será alcançado com folga de '+str(round(dif,2))+' reais. A parcela mínima necessária para você alcançar seu objettivo é '+str(round(PMTt,2))+' reais.'}

  controle_parcela.update(aux)

  with open(salvar_em+'/meta_parcela.json', 'wb') as fp:
    pickle.dump(controle_parcela, fp)



os arquivos serão salvos em: /content/sample_data/Clientes do Dia
Foram encontrados 1356 fundos da sua instituição
Foram encontrados 4919 fundos da sua instituição
Foram encontrados 2189 fundos da sua instituição
os arquivos serão salvos em: /content/sample_data/Clientes do Dia
os arquivos serão salvos em: /content/sample_data/Clientes do Dia
os arquivos serão salvos em: /content/sample_data/Etapa Final


#objetivo é alcançável?

In [ ]:
base_historico = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Carteira Quant/Base Dados/Base_Original.csv",index_col=1)

In [ ]:
cnpjs_mais500 = base_historico.loc[base_historico['DT_COMPTC'] == base_historico['DT_COMPTC'].drop_duplicates()[-2]].loc[base_historico.loc[base_historico['DT_COMPTC'] == base_historico['DT_COMPTC'].drop_duplicates()[-2]]['NR_COTST'] > 500].index
base_historico = base_historico.loc[cnpjs_mais500]

In [ ]:
cadastrais = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Carteira Quant/Base Dados/base_cliente.csv",
                         low_memory=False,index_col=0)

In [ ]:
bases_consulta = {}
for cliente in clientes_dia.index:
  contidos = []
  for cnpj in cnpjs_clientes[cliente]:
    if cnpj[0] in cadastrais.index:
      
      contidos.append(cnpj[0])
    else:
      pass
    
  df_aux = cadastrais.loc[contidos]

  df_aux = df_aux.loc[df_aux['INVEST_INICIAL_MIN'] < float(clientes_dia.loc[cliente]['Quanto você investirá por mês?'])]
  df_aux = df_aux.loc[df_aux['SIT'] != 'CANCELADA']
  df_aux = df_aux.loc[df_aux['INVEST_QUALIF'] != 'S']
  
  categoria = clientes_dia.loc[cliente]['Em qual categoria o seu objetivo mais se enquadra?']
  if categoria == 'Aposentadoria':
    df_aux = df_aux.loc[df_aux['PUBLICO_ALVO'] == 'PREVIDENCIÁRIO']
  
  else:
    pass
  
  try:
    fis = base_historico.loc[df_aux.index].groupby(['CNPJ_FUNDO','DT_COMPTC']).mean()['VL_QUOTA']
  
  except:
    contidos = []
    for cnpj in df_aux.index:
      if cnpj in (base_historico.index):
        contidos.append(cnpj)
      else:
        pass
    
    fis = base_historico.loc[contidos].groupby(['CNPJ_FUNDO','DT_COMPTC']).mean()['VL_QUOTA']
  
  valor_quota = pd.DataFrame()
  for fundo in fis.index.get_level_values(0).drop_duplicates():
    valor_quota[fundo] = fis.loc[fundo]
  

  valor_quota.index = pd.to_datetime(valor_quota.index)
  
  criterios = pd.DataFrame()

  drawdowns = (1 - ((1+(valor_quota.pct_change())).cumprod()-1).div(((1+(valor_quota.pct_change())).cumprod()-1).cummax()))*-1 #gera o drawdown de cada fundo
  
  criterios['Drawdown Medio'] = drawdowns.mean()
  
  criterios['Drawdown CP'] = drawdowns.rolling(66).mean().mean()
  
  criterios['Sharpe'] = sharpe = (((1+(valor_quota.pct_change())).rolling(66).apply(np.prod)-1).mean() - ((1+(cdi['valor']/100)).rolling(66).apply(np.prod)-1).mean())/((1+(valor_quota.pct_change())).rolling(66).apply(np.prod)-1).std()
    
  criterios = criterios.loc[criterios['Drawdown CP'] > (int(clientes_dia.loc[cliente]['Qual o valor máximo de perdas que toleraria num curto prazo?'])/100)*-1]
                                                                                
  criterios = criterios.loc[criterios['Sharpe'] > 0]                           

  criterios = criterios.loc[criterios['Drawdown CP'] > criterios['Drawdown CP'].median() ]

  bases_consulta.update({cliente:valor_quota[criterios.index]})

with open(salvar_em+'/CNPJS_SOLICITANTES.json', 'wb') as fp:
  pickle.dump(bases_consulta, fp)